Adicionar o path de Dados do Google Drive

In [1]:
import plotly.express as px
import pandas as pd
import gspread
import glob

Carregar os Dados da Planilha online Google Sheets

In [2]:
class GoogleSheets:
    def __init__(self):
        self.key = glob.glob("key/*.json")[0]
        self.CODE = '1AYpQ_uHm7aZ-VZwibuscpQxgJ-vXirm4PWrJDECYr1c'
        self.gc = gspread.service_account(filename=self.key)
        self.sh = self.gc.open_by_key(self.CODE)

    def visualizar_dados(self):
        ws = self.sh.worksheet('CONVIDADOS DO CASAMENTO')
        dados = ws.get_all_records()
        planilha = pd.DataFrame(dados)
        return planilha
    
    def transformar_minuscula(self,coluna):
        coluna_capitalizada = coluna.str.title()
        return coluna_capitalizada
    
# Create an instance of the GoogleSheets class
googlesheet = GoogleSheets()

df=googlesheet.visualizar_dados()
df["Nome"] = googlesheet.transformar_minuscula(df["Nome"])
df["Convite Enviado ?"] = df["Convite Enviado ?"].fillna("Não informado")
df

,Nome,PARENTESCO:,Parentesco,Padrinho,Localidade:,Criança,Convite Enviado ?,Codigo,Pessoas da Fazenda
0,Elenice Inês Dos Santos,MAE DO NOIVO,Mãe do noivo,,,Não,Sim,64ddc,12
1,Luiz Felipe Linhares,,Amigo do noivo,,,Não,Sim,6606c,
2,Verônica,,Mulher Felipe Linhares,,,Não,Sim,673c1,
3,Ana Luiza Dos Santos Moreira,IRMA DO NOIVO,Madrinha do noivo,Sim,,Não,Sim,6894f,
4,Lucas Junior Dos Santos Moreira,IRMAO DO NOIVO,Irmão do noivo,,,Não,Sim,69eef,
...,...,...,...,...,...,...,...,...,...
152,Marcia Lopes Coelho,,Mulher do pastor que vai realizar o casamento,,,Não,Sim,8841f,
153,Pastor Cláudio,,O pastor que fazer o casamento,,,Não,Sim,f3cf5,
154,Luiz Paulo,,Amigo do noivo,,,Não,Sim,f899,
155,Vanessa Gomes,,Amigo do noivo,,,Não,Sim,2359y,


Criar vizualização para ver os convites enviados

In [ ]:
from plotly.subplots import make_subplots

import plotly.graph_objs as go

# Criar subplots com 2 linhas e 2 colunas
fig = make_subplots(rows=2, cols=2, subplot_titles=('Convites Enviados', 'Parentesco', 'Localidade', 'Padrinho'), specs=[[{'type':'pie'}, {'type':'pie'}], [{'type':'pie'}, {'type':'pie'}]])

# Gráfico 1: Convites Enviados
convites_enviados = df['Convite Enviado ?'].value_counts().reset_index()
convites_enviados.columns = ["Convite Enviado ?", "Quantidade"]
fig.add_trace(go.Pie(labels=convites_enviados['Convite Enviado ?'], values=convites_enviados['Quantidade'], hole=0.6), row=1, col=1)

# Gráfico 2: Parentesco
parentesco = df['Parentesco'].value_counts().reset_index()
parentesco.columns = ["Parentesco", "Quantidade"]
fig.add_trace(go.Pie(labels=parentesco['Parentesco'], values=parentesco['Quantidade'], hole=0.6), row=1, col=2)

# Gráfico 3: Localidade
localidade = df['Localidade:'].value_counts().reset_index()
localidade.columns = ["Localidade", "Quantidade"]
fig.add_trace(go.Pie(labels=localidade['Localidade'], values=localidade['Quantidade'], hole=0.6), row=2, col=1)

# Gráfico 4: Padrinho
padrinho = df['Padrinho'].value_counts().reset_index()
padrinho.columns = ["Padrinho", "Quantidade"]
fig.add_trace(go.Pie(labels=padrinho['Padrinho'], values=padrinho['Quantidade'], hole=0.6), row=2, col=2)

# Atualizar layout
fig.update_layout(title_text='Visualizações dos Dados dos Convites', showlegend=False)

fig.show()

Gerar os Convites

In [5]:
from PIL import Image, ImageDraw, ImageFont
import shutil
import os
import pandas as pd
import uuid


# Classe do Sistema
class Sistema:
    @staticmethod
    def apagar_conteudo_diretorio(diretorio):
        if os.path.exists(diretorio):
            for item in os.listdir(diretorio):
                item_path = os.path.join(diretorio, item)
                try:
                    if os.path.isfile(item_path) or os.path.islink(item_path):
                        os.unlink(item_path)  # Remove arquivos ou links simbólicos
                    elif os.path.isdir(item_path):
                        shutil.rmtree(item_path)  # Remove diretórios recursivamente
                except Exception as e:
                    print(f'Erro ao apagar {item_path}: {e}')
            print(f'Todo o conteúdo do diretório {diretorio} foi apagado com sucesso.')
        else:
            print(f'O diretório {diretorio} não existe.')

# Exemplo de uso
diretorio_para_limpar = '/content/drive/MyDrive/Colab Notebooks/GeradorConvitesCasamento/convites_gerados'
Sistema.apagar_conteudo_diretorio(diretorio_para_limpar)

# Definir fontes
font1 = ImageFont.truetype("font/font.otf", size=45)
font2 = ImageFont.truetype("font/font.otf", size=30)
path_imagem_base = "template/template.png"
path_imagem_output = "convites_gerados"

def gerar_convite(convidado, codigo,path_imagem_base,path_imagem_output,font1,font2):

    imagem = Image.open(path_imagem_base).convert("RGBA")

    lapis = ImageDraw.Draw(imagem)

    dados_imagem = imagem.size
    dados_texto = len(convidado)

    print(f"Pessoa: {convidado}, Tamanho do texto: {dados_texto}")

    # Calcular a posição x para centralizar o texto
    # Use getbbox instead of getsize
    posicao_x = (dados_imagem[0] - font1.getbbox(convidado)[2]) // 2

    # Nome do Convidado
    lapis.text((posicao_x, 1170), text=convidado, fill="#000", font=font1) # Update to use posicao_x

    # Código do Convite
    lapis.text((165, 1265), text=f"código: {codigo}", fill="#000", font=font2)

    # Salvar a imagem com o nome do convidado.
    imagem.save(f"{path_imagem_output}/CONVITE - {convidado}.png")

for convidado in df.values:
    gerar_convite(convidado[0], convidado[7],path_imagem_base,path_imagem_output,font1,font2)

#lista_df.to_excel("/content/drive/MyDrive/Colab Notebooks/GeradorConvitesCasamento/convites_gerados/LISTA.xlsx", index=False)

O diretório /content/drive/MyDrive/Colab Notebooks/GeradorConvitesCasamento/convites_gerados não existe.
Pessoa: Elenice  Inês Dos Santos, Tamanho do texto: 24
Pessoa: Luiz Felipe Linhares, Tamanho do texto: 20
Pessoa: Verônica , Tamanho do texto: 9
Pessoa: Ana Luiza Dos Santos Moreira , Tamanho do texto: 29
Pessoa: Lucas Junior Dos Santos Moreira  , Tamanho do texto: 33
Pessoa: Marcos Santos, Tamanho do texto: 13
Pessoa: Sandro, Tamanho do texto: 6
Pessoa: Mateus Francisco, Tamanho do texto: 16
Pessoa: Miguel, Tamanho do texto: 6
Pessoa: Jucilene , Tamanho do texto: 9
Pessoa: Andréia De Fátima Santos, Tamanho do texto: 24
Pessoa: Carlos André Dos Santos, Tamanho do texto: 23
Pessoa: Joesia Inês De Souza  Rodrigues , Tamanho do texto: 32
Pessoa: Ricardo Rodrigues , Tamanho do texto: 18
Pessoa: Ana Lívia , Tamanho do texto: 10
Pessoa: Josimar Marcela Dos Santos Silva, Tamanho do texto: 32
Pessoa: Genésio Antônio Da Silva, Tamanho do texto: 24
Pessoa: Bruno Vitor Da Silva, Tamanho do tex

KeyboardInterrupt: 